<a href="https://colab.research.google.com/github/ernie-Griffiths/Determining-the-Validity-of-Genetically-Engineered-Crops-in-a-Warming-Climate-Using-Machine-Learning/blob/main/Radial_Basis_Function_NN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler

from keras.models import Sequential

from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

from sklearn.model_selection import train_test_split

from keras.callbacks import ModelCheckpoint

from keras.layers import Layer
from keras import backend as K

import matplotlib.pyplot as plt


In [ ]:
Phenotype_Soil_Weather_Data = pd.read_csv("/content/drive/MyDrive/University of reading/MSC FINAL project/Data/Data Files/Finalised data/FeatureSelected_Phenotype_Soil_Weather_Data.csv",index_col=[0])

In [ ]:
Phenotype_Soil_Weather_Data['Texture'] = pd.factorize(Phenotype_Soil_Weather_Data['Texture'])[0] + 1
Phenotype_Soil_Weather_Data['Seed Success factor'] = Phenotype_Soil_Weather_Data['Seed Success factor'].round(decimals = 3)
Phenotype_Soil_Weather_Data = Phenotype_Soil_Weather_Data.dropna()
Phenotype_Soil_Weather_Data.head(2)

,Field-Location,Source,Pedigree,Family,Tester,Replicate,Block,Range,Pass,Anthesis [MM/DD/YY],...,Temperature [C],Dew Point [C],Relative Humidity [%],Solar Radiation [W/m2],Rainfall [mm],Wind Speed [m/s],Wind Direction [degrees],Wind Gust [m/s],Soil Temperature [C],Seed Success factor
0,DEH1,WIPV17/30359,PHN11_PHW65_0512/PHT69,PHN11_PHW65,PHT69,1.0,1,2.0,2.0,7/2/2019,...,23.041976,13.951074,72.528273,212.490939,0.012235,0.22562,141.449484,6.775455,23.81826,1.525
1,DEH1,WIPV17/30278,PHN11_PHW65_0278/PHT69,PHN11_PHW65,PHT69,1.0,1,2.0,3.0,7/2/2019,...,23.041976,13.951074,72.528273,212.490939,0.012235,0.22562,141.449484,6.775455,23.81826,2.057


In [ ]:
cols_to_norm = ['Anthesis [days]', 'Silking [days]', 'Plant Height [cm]', 'Ear Height [cm]', 'Stand Count [# of plants]', 'Grain Moisture [%]', 'Test Weight [lbs]', 'Plant Month','Harvest Month','E Depth','1:1 Soil pH','WDRF Buffer pH','1:1 S Salts mmho/cm','Texture No','Organic Matter LOI %','Nitrate-N ppm N','lbs N/A','Potassium ppm K','Sulfate-S ppm S','Calcium ppm Ca','Magnesium ppm Mg','Sodium ppm Na','CEC/Sum of Cations me/100g','%H Sat','%K Sat','%Ca Sat','%Mg Sat','%Na Sat','Mehlich P-III ppm P','% Sand','% Silt','% Clay','Texture','Temperature [C]','Dew Point [C]','Relative Humidity [%]','Solar Radiation [W/m2]','Rainfall [mm]','Wind Speed [m/s]','Wind Direction [degrees]','Wind Gust [m/s]','Soil Temperature [C]','Seed Success factor']
Phenotype_Soil_Weather_Data[cols_to_norm] = StandardScaler().fit_transform(Phenotype_Soil_Weather_Data[cols_to_norm])

In [ ]:
xdata = Phenotype_Soil_Weather_Data.drop(["Seed Success factor","Field-Location",	"Source","Pedigree",	"Family",	"Tester",	"Replicate",	"Block","Anthesis [MM/DD/YY]","Silking [MM/DD/YY]","Location"], axis = 1)
ydata = Phenotype_Soil_Weather_Data["Seed Success factor"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(xdata, ydata, test_size = 0.4,shuffle = True, random_state = 42)

In [ ]:
#https://stackoverflow.com/questions/53855941/how-to-implement-rbf-activation-function-in-keras
#BUilding a RBF model
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)


    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
            "gamma": self.gamma,
        })
        return config

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [ ]:
model = Sequential()
# Input layer
model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))
# Hidden layer
model.add(RBFLayer(256, 0.1))
#Output layer.
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

In [ ]:
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5760      
                                                                 
 rbf_layer (RBFLayer)        (None, 256)               32768     
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 38,785
Trainable params: 38,785
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint_name = 'BestWeight.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
101/104 [============================>.] - ETA: 0s - loss: 0.7781 - mean_absolute_error: 0.7781
Epoch 1: val_loss improved from inf to 0.70718, saving model to BestWeight.hdf5
104/104 [==============================] - 3s 20ms/step - loss: 0.7762 - mean_absolute_error: 0.7762 - val_loss: 0.7072 - val_mean_absolute_error: 0.7072
Epoch 2/500
103/104 [============================>.] - ETA: 0s - loss: 0.6285 - mean_absolute_error: 0.6285
Epoch 2: val_loss improved from 0.70718 to 0.60658, saving model to BestWeight.hdf5
104/104 [==============================] - 2s 16ms/step - loss: 0.6276 - mean_absolute_error: 0.6276 - val_loss: 0.6066 - val_mean_absolute_error: 0.6066
Epoch 3/500
101/104 [============================>.] - ETA: 0s - loss: 0.5843 - mean_absolute_error: 0.5843
Epoch 3: val_loss improved from 0.60658 to 0.55669, saving model to BestWeight.hdf5
104/104 [==============================] - 2s 16ms/step - loss: 0.5823 - mean_absolute_error: 0.5823 - val_loss: 0.5567 

In [ ]:
# Load weights file of the best model :
weights_file = '/content/BestWeight.hdf5' # choose the best checkpoint
model.load_weights(weights_file) # load it
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
predictions = model.predict(x_test)
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])#
acc_score = r2_score(y_test, predictions)
Mean_Error = mean_squared_error(y_test, predictions, squared=False)
print("Accuracy score")
print(acc_score)
print("Error Bracket")
print(Mean_Error)

Results = pd.read_csv("/content/drive/MyDrive/University of reading/MSC FINAL project/Data/Data Files/Results /LAResults.csv",index_col=[0])

dict = {'Learning Algorithum': 'Radial Basis Function NN', 'R2 Score': acc_score, 'Error Bracket': Mean_Error}

Results = Results.append(dict, ignore_index = True)


Accuracy score
0.722956539547114
Error Bracket
0.5280652362438263


In [ ]:
Results

,Learning Algorithum,R2 Score,Error Bracket
0,Regression Tree,0.671146,0.332852
1,Random Forest,0.720002,0.303280
2,KNN Regression,0.693163,0.318367
3,Gradient Boosting Regressor,0.741307,0.295217
4,Convolutional Nerual Network,0.696746,0.319638
5,MLP Regressor,0.723194,0.305378
6,Radial Basis Function NN,0.722957,0.528065


In [ ]:

# Results.to_csv('LAResults.csv')
# !cp LAResults.csv "/content/drive/MyDrive/University of reading/MSC FINAL project/Data/Data Files/Results "

In [ ]:
predictions

array([[ 0.12647103],
       [-0.23645157],
       [-0.5921656 ],
       ...,
       [ 1.1191313 ],
       [-0.04042824],
       [ 0.00535872]], dtype=float32)